In [1]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import seaborn as sns
import geopandas as gpd
#import osmnx as ox
from shapely.geometry import Point
import statsmodels.api
from scipy.stats import chi2_contingency
from scipy.stats import pearsonr

import warnings
warnings.filterwarnings('ignore')

# Chemins d'accès aux données
repDataConsolidees = '../../dataConsolidees.gouv/'

# Liste des fichiers usagers disponibles
!pwd
!ls {repDataConsolidees}*  -lrt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


/Users/stephane/Documents/Professionnel/Formation/Certificats/Datascientest/4 - Projet AccidentsRoutiers/scripts/1 - ExplorationDonnees
ls: -lrt: No such file or directory
../../dataConsolidees.gouv/agg_accidents.csv
../../dataConsolidees.gouv/agg_usagers.csv
../../dataConsolidees.gouv/test_set.csv
../../dataConsolidees.gouv/train_set.csv
../../dataConsolidees.gouv/usagers.csv


In [2]:
# ETAPE 1 - Chgt des données 
train = pd.read_csv (repDataConsolidees + "train_set.csv", dtype={'user_actp' : str}, ) 
train.describe()


,id_accident,id_vehicule,carac_an,carac_mois,carac_jour,carac_hrmn,carac_agg,carac_atm,carac_col,carac_com,carac_dept,carac_gps_lat,carac_gps_long,carac_int,carac_lum,lieu_catr,lieu_circ,lieu_infra,lieu_larrout,lieu_lartpc,lieu_nbv,lieu_plan,lieu_prof,lieu_situ,lieu_surf,lieu_vma,lieu_vosp,vehi_catv,vehi_choc,vehi_manv,vehi_motor,vehi_obs,vehi_obsm,vehi_occutc,vehi_senc,user_an_nais,user_catu,user_etatp,user_locp,user_place,user_secu1,user_secu2,user_secu3,user_sexe,user_trajet,user_gravite,agg_catv_perso,agg_is_conducteur_vehicule,agg_is_conductrice_vehicule,agg_nb_pieton_vehicule,agg_nb_passager_vehicule,agg_nb_indemne_vehicule,agg_nb_blesse_leger_vehicule,agg_nb_blesse_grave_vehicule,agg_nb_tue_vehicule,agg_nb_total_vehicule,agg_nb_total_velo,agg_nb_total_vsp,agg_nb_total_moto,agg_nb_total_vl,agg_nb_total_pl,agg_nb_total_va,agg_nb_total_conducteur,agg_nb_total_conductrice,agg_nb_total_pieton,agg_nb_total_passager
count,2.709320e+05,2.709320e+05,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,266472.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000,270932.000000
mean,2.021057e+11,4.553213e+07,2021.056287,6.744729,15.671213,1388.243209,1.620827,1.601959,3.711330,203.709451,110.517170,44.236497,2.495289,2.060377,1.889884,3.294982,1.830349,0.788862,-0.577675,2.451715,2.451715,1.287485,1.231047,1.517757,1.254979,60.475411,0.205302,12.588435,2.864830,7.041597,1.232918,1.127833,1.582604,0.062012,1.561643,1982.778994,1.328437,-0.830895,0.239352,2.071605,1.856189,0.995169,-0.915414,1.277446,2.945842,0.782754,3.668374,0.743467,0.253071,0.167105,0.564636,0.747804,0.689775,0.247402,0.043299,1.896974,0.090495,0.181134,0.145561,1.347179,0.077536,0.055069,1.409741,0.482490,0.171578,0.817903
std,7.985274e+07,7.015165e+07,0.798512,3.351423,8.746800,541.783540,0.485182,1.672645,1.909646,177.334310,212.904488,12.433591,19.810026,1.992136,1.468990,1.290287,0.906522,2.149915,1.850557,1.406757,1.406757,0.666211,0.544423,1.478241,0.808818,25.812258,0.664097,13.322957,2.398627,8.030516,1.018827,3.254549,1.227709,1.073001,0.814661,18.830455,0.609379,0.614434,0.990822,2.566376,2.259698,3.055236,0.891649,0.545001,3.086431,0.796857,0.988822,0.436720,0.434772,0.458028,1.415123,1.222352,0.920268,0.532908,0.229058,0.835856,0.350791,0.401643,0.369411,0.955718,0.295065,0.234766,0.862279,0.651581,0.468361,1.700110
min,2.020000e+11,1.008820e+05,2020.000000,1.000000,1.000000,0.000000,1.000000,-1.000000,-1.000000,1.000000,1.000000,-23.868998,-178.158090,-1.000000,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,1900.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.020000e+11,1.710668e+05,2020.000000,4.000000,8.000000,1010.000000,1.000000,1.000000,2.000000,68.000000,34.000000,44.629000,1.374573,1.000000,1.000000,3.000000,2.000000,0.000000,-1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,50.00